# TurtleTrader Validatie & Verificatie Notebook

Dit uitgebreide validatienotebook is ontworpen om de TurtleTrader-strategie grondig te testen voordat we echt kapitaal inzetten. We voeren de volgende validaties uit:

1. MT5-verbinding en data-ophaling
2. Datakwaliteitsbeoordeling (gaten, uitschieters, integriteit)
3. Validatie van strategie-indicatorberekeningen
4. Identificatie van handelssignalen
5. Realistische backtesting met transactiekosten en slippage
6. Gedetailleerde drawdown-analyse met duurmetrieken
7. Robuuste parameteroptimalisatie met in-sample en out-of-sample tests
8. Prestatievergelijking tussen verschillende marktomstandigheden

In [13]:
# CODE CEL - 1: Imports en configuratie
import sys
import os
import json
import traceback
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import stats
import MetaTrader5 as mt5
from sklearn.model_selection import TimeSeriesSplit

# Add the project path to the system path to enable imports
# Determine the absolute path to the project root
notebook_dir = os.path.abspath('')
project_path = os.path.abspath(os.path.join(notebook_dir, '..'))
print(f"Using project path: {project_path}")
sys.path.append(project_path)

# Import TurtleTrader modules
try:
    from modules.mt5_connector import MT5Connector
    from modules.strategy import TurtleStrategy
    from modules.risk_manager import RiskManager
    print("✅ TurtleTrader modules successfully imported")
except ImportError as e:
    print(f"❌ Error importing TurtleTrader modules: {e}")
    print("Ensure your project path is correct and all required modules exist")

# Configure visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['lines.linewidth'] = 1.5
sns.set_style("whitegrid")

# Enable error stack trace display
def display_error(e):
    """Display detailed error information"""
    print(f"\n❌ ERROR: {type(e).__name__}: {e}")
    traceback.print_exc()
    print("\nPlease check the error details above and fix the issue before continuing.")

Using project path: C:\Users\basti\PycharmProjects
✅ TurtleTrader modules successfully imported


In [14]:
# CODE CEL - 1: Imports en configuratie
import sys
import os
import json
import traceback
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import stats
import MetaTrader5 as mt5
from sklearn.model_selection import TimeSeriesSplit

# Add the project path to the system path to enable imports
# Adjust this to your project path if needed
project_path = "../.."
sys.path.append(project_path)

# Import TurtleTrader modules
try:
    from modules.mt5_connector import MT5Connector
    from modules.strategy import TurtleStrategy
    from modules.risk_manager import RiskManager
    print("✅ TurtleTrader modules successfully imported")
except ImportError as e:
    print(f"❌ Error importing TurtleTrader modules: {e}")
    print("Ensure your project path is correct and all required modules exist")

# Configure visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['lines.linewidth'] = 1.5
sns.set_style("whitegrid")

# Enable error stack trace display
def display_error(e):
    """Display detailed error information"""
    print(f"\n❌ ERROR: {type(e).__name__}: {e}")
    traceback.print_exc()
    print("\nPlease check the error details above and fix the issue before continuing.")

✅ TurtleTrader modules successfully imported


# CODE CEL - 3: Connect to MT5
# Connect to MT5 using the project's connector

In [16]:
# CODE CEL - 2: Load configuration
# Load settings from the TurtleTrader config file
def load_config(config_path):
    """Load configuration from JSON file with error handling"""
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        raise FileNotFoundError(f"Configuration file not found at {config_path}. Please check the path.")
    except json.JSONDecodeError:
        raise ValueError(f"Invalid JSON in configuration file {config_path}. Please check the file format.")

try:
    # Use the notebook directory as the project directory since the notebook is already in the project root
    notebook_dir = os.path.abspath('')
    project_dir = notebook_dir  # Don't go up a level, we're already in the project root

    print(f"Notebook directory: {notebook_dir}")
    print(f"Project directory: {project_dir}")

    # Build the path to the configuration file
    config_path = os.path.join(project_dir, 'config', 'turtle_settings.json')
    print(f"Looking for configuration at: {config_path}")

    # Check if the file exists
    if not os.path.exists(config_path):
        print(f"⚠️ Configuration file not found at the expected location.")
        print(f"Let's check the directory structure:")

        # List directories to help debugging
        project_contents = os.listdir(project_dir)
        print(f"Contents of {project_dir}:")
        for item in project_contents:
            if os.path.isdir(os.path.join(project_dir, item)):
                print(f"  📁 {item}/")
            else:
                print(f"  📄 {item}")

    # Load the configuration
    config = load_config(config_path)

    print("TurtleTrader Configuration:")
    print(f"Broker: {config['mt5']['server']}")
    print(f"Symbols: {config['mt5']['symbols']}")
    print(f"Risk per trade: {config['mt5']['risk_per_trade'] * 100}%")
    if 'symbol_mapping' in config['mt5']:
        print(f"Symbol mapping: {config['mt5']['symbol_mapping']}")
except Exception as e:
    display_error(e)

Notebook directory: C:\Users\basti\PycharmProjects\TurtleTrader
Project directory: C:\Users\basti\PycharmProjects\TurtleTrader
Looking for configuration at: C:\Users\basti\PycharmProjects\TurtleTrader\config\turtle_settings.json
TurtleTrader Configuration:
Broker: FTMO-Demo2
Symbols: ['EURUSD', 'XAUUSD', 'US30']
Risk per trade: 1.0%
Symbol mapping: {'US30': 'US30.cash'}
